In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
today = date.today()
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')
print(f'💚 Today is {today}')
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))
import ss_api_call as ss

📂 Current working directory: /Users/serenekim/Desktop/PhD/meta-wealth_mobility
💚 Today is 2025-10-03


In [3]:
df = pd.read_csv(f'{path}/data_abstracts/true_mobility_studies_617_forKGs_cleaned.csv')

In [4]:
df['period'] = pd.cut(df['year'], bins=[1900, 2000, 2005, 2010, 2015, 2020, 2025], right=True, labels=["-2000", "2001-2005", "2006-2010", "2011-2015", "2016-2020", "2021-2025"])
df['period'].value_counts().sort_index()

period
-2000         23
2001-2005     23
2006-2010     65
2011-2015    141
2016-2020    183
2021-2025    182
Name: count, dtype: int64

In [12]:
data_cat_num = {
    "Panel/Longitudinal Surveys": 1,
    "Administrative/Registry Data": 2, 
    "National Survey Data": 3,
    "Opportunity Atlas": 4,
    "Natural/Experimental Data": 5,
    "Linked Administrative Data": 6,
    "International Panel Data": 7,
    "Rich List Data": 8,
    "University/Institution Data": 9,
    "Pseudo-Panel/Household Budget Survey": 10,
    "Archival/Historical Data": 11,
    "Big Data": 12,
    "No dataset": 13,
    "Others": 14
}

rq_cat_num = {
    "Measurement and Methodological Advances": 1,
    "Empirical Estimates and Determinants": 2,
    "Policy, Institutional, and Geographic Impacts": 3,
    "Intergenerational Wealth Mobility and Inheritance": 4,
    "Demographic Differences in Mobility (Race, Gender, etc.)": 5,
    "Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)": 6,
    "Theoretical and Structural Models": 7,
    "Perceptions of Mobility and Attitudes": 8,
    "Others": 9
}
    
    
    
df['data_cat_num'] = df['data_cat'].replace(data_cat_num).astype(int)
df['rq_cat_num'] = df['rq_cat'].replace(rq_cat_num).astype(int)

/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_49250/513924957.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['data_cat_num'] = df['data_cat'].replace(data_cat_num).astype(int)
/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_49250/513924957.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rq_cat_num'] = df['rq_cat'].replace(rq_cat_num).astype(int)


In [15]:
df.columns

Index(['id', 'title', 'year', 'doi', 'landing_page', 'abstract_inverted_index',
       'language', 'is_oa', 'oa_status', 'oa_link', 'abstract', 'abstract_sm',
       'authors_sm', 'domain', 'sort_gpt_1', 'sort_gpt_2', 'sort_gpt4o_1',
       'sort_gpt4o_2', 'cited_by', 'len_cited_by', 'ref_count',
       'cited_by_count', 'Q1', 'Q1_1', 'Q2', 'Q2_1', 'Q2_2', 'Q3', 'Q4', 'abs',
       'index', 'category_n1', 'measure', 'specified', 'relevant',
       'category_n2', 'measure_1', 'measure_2', 'category_3', 'file', 'id.1',
       'Question1', 'Question2', 'rq_cat', 'RQ', 'data_cat', 'data_type',
       'type', 'category_1', 'category_2', 'author_raw_names',
       'author_raw_affiliations', 'topic_display_names', 'topic_scores',
       'field_display_names', 'subfield_display_names', 'period',
       'data_cat_num', 'rq_cat_num'],
      dtype='object')

In [17]:
df[['category_n1', 'data_cat_num', 'rq_cat_num']]

,category_n1,data_cat_num,rq_cat_num
0,4,6,4
1,1,13,2
2,1,13,2
3,3,1,1
4,4,14,2
...,...,...,...
612,1,1,2
613,1,13,7
614,7,1,4
615,6,13,7


# how the families are cited (Appendix; Or don't do this?)
* how many connected components? --> hops. 
* different directions of a pair can be compared (structure A cites B vs. B cites A)

In [19]:
import pandas as pd
from neo4j import GraphDatabase

# --- Connect ---
driver = GraphDatabase.driver("bolt://localhost:7690", auth=("neo4j", "your_password"))

# Normalize categories
df['category_1'] = df['category_1'].replace({'Others': 'Others_Measure'})
df['category_2'] = df['category_2'].replace({'Others': 'Others_Measure'})
df['data_cat']   = df['data_cat'].replace({'Others': 'Others_DataType'})
df['rq_cat']     = df['rq_cat'].replace({'Others': 'Others_RqType'})

def safe_str(val):
    if pd.isna(val) or str(val).strip().lower() in {"", "nan", "none"}:
        return None
    return str(val).strip()

# --- Constraints ---
with driver.session() as session:
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Measure)   REQUIRE m.name IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (ds:DataType) REQUIRE ds.name IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (r:RqType)    REQUIRE r.name IS UNIQUE")

# --- Collect rows ---
rows = []
for _, row in df.iterrows():
    m1       = safe_str(row.get("category_1"))
    # m2       = safe_str(row.get("category_2"))
    datatype = safe_str(row.get("data_cat"))
    rqtype   = safe_str(row.get("rq_cat"))
    period = row.get("period")
    paper_id = safe_str(row.get("id"))
    m1_num = row.get("category_n1")
    datatype_num = row.get("data_cat_num")
    rqtype_num = row.get("rq_cat_num")
    

    if not all([m1, datatype, rqtype, period]):
        continue  # skip incomplete rows

    rows.append({
        "m1": m1,
        # "m2": m2,
        "datatype": datatype,
        "rqtype": rqtype,
        "period": period,
        "paper_id": paper_id,
        "m1_num": m1_num,
        "datatype_num": datatype_num,
        "rqtype_num": rqtype_num
    })

# --- Cypher with undirected edges ---
cypher = """
UNWIND $rows AS row

MERGE (m1:Measure {name: row.m1})
MERGE (ds:DataType {name: row.datatype})
MERGE (rq:RqType {name: row.rqtype})

// Measure -- DataType
MERGE (m1)-[r1:CO_MEASURE_DATATYPE {period: row.period}]-(ds)
  ON CREATE SET r1.w = 1, r1.papers=[row.paper_id]
  ON MATCH  SET r1.w = r1.w + 1, r1.papers = apoc.coll.toSet(coalesce(r1.papers, []) + row.paper_id)

// DataType -- RqType
MERGE (ds)-[r2:CO_DATATYPE_RQTYPE {period: row.period}]-(rq)
  ON CREATE SET r2.w = 1, r2.papers=[row.paper_id]
  ON MATCH  SET r2.w = r2.w + 1, r2.papers = apoc.coll.toSet(coalesce(r2.papers, []) + row.paper_id)

// RqType -- Measure
MERGE (rq)-[r3:CO_RQTYPE_MEASURE {period: row.period}]-(m1)
  ON CREATE SET r3.w = 1, r3.papers=[row.paper_id]
  ON MATCH  SET r3.w = r3.w + 1, r3.papers = apoc.coll.toSet(coalesce(r3.papers, []) + row.paper_id)

// --- Supernode for family ---
WITH m1, ds, rq, row
WITH row,
     "M" + row.m1_num + "-D" + row.datatype_num + "-R" + row.rqtype_num AS family_name,
     m1, ds, rq

MERGE (f:Family {name: family_name})
  ON CREATE SET f.w = 1, f.papers=[row.paper_id]
  ON MATCH  SET f.w = f.w + 1, f.papers = apoc.coll.toSet(coalesce(f.papers, []) + row.paper_id)

// Connect Family to its triplet (optional, if you want structure visible)
MERGE (f)-[:HAS_MEASURE]-(m1)
MERGE (f)-[:HAS_DATATYPE]-(ds)
MERGE (f)-[:HAS_RQTYPE]-(rq)
"""


with driver.session() as session:
    if rows:
        session.run(cypher, rows=rows)

driver.close()
